#Pre made API form
rapidAPI: https://rapidapi.com/LukeDevelops/api/ai-content-identifier2/playground/apiendpoint_68656d8e-1dc6-42b0-b207-8e5493f90a2f

HuggingFace Space: https://eeshaharshad-aichecker.hf.space






Installing Important Libraries

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!ngrok config add-authtoken 2fNQ7AEiwJ6tGGOxYyReXF9rFKu_6Pmy7vMkJCuNYQiesnH8S
!pip install python-multipart
!pip install langsmith
!pip install langchain-community
!pip install pypdf
!pip install langchain fastapi
!pip install -U Langsmith
!pip install "unstructured[docx]"

Importing Necessary Libraries

In [2]:
import os
import tempfile
import fastapi
import uvicorn
import requests
import pandas
import langchain
import langsmith as ls
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI, Request, Form
from fastapi.responses import JSONResponse
from fastapi import FastAPI, File, UploadFile, HTTPException
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain_core.tracers.context import tracing_v2_enabled
from langsmith import traceable

Setting up Monitoring Environment

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AiChecker"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"

Main Code Body

In [4]:
app = FastAPI()

# API key and host
API_KEY = "c22f916d7emsh84b9036bb10a1b9p1f1cecjsne5deda81b82a"
API_HOST = "ai-content-identifier2.p.rapidapi.com"

#using traceable decorator for langsmith tracing
@ls.traceable
# Code for Text AI Detection endpoint
def text_check(user_input):
    url = "https://ai-content-identifier2.p.rapidapi.com/text"
    payload = {
        "text": user_input,
        "threshold": 10  # Adjust this if needed to test different sensitivity levels
    }
    headers = {
        "x-rapidapi-key": API_KEY,
        "x-rapidapi-host": API_HOST,
        "Content-Type": "application/json"
    }
    response = requests.post(url, json=payload, headers=headers)
    return response.json()


@app.get('/')
async def home():
    return "Hello! Welcome to Eshah's Final Project"

@ls.traceable
@app.post("/uploadFile/")
async def create_upload_files(file: UploadFile = File(...)):
    suffix = os.path.splitext(file.filename)[1].lower()
    temp_file_path = tempfile.mktemp(suffix=suffix)
    with open(temp_file_path, 'wb') as f:
        f.write(await file.read())

    try:
        if suffix == '.pdf':
            loader = PyPDFLoader(temp_file_path)
            document = loader.load_and_split()
            print(dir(document[0])) if document else print("No document loaded") ## Debugging
            text_content = " ".join([str(page) for page in document])
        elif suffix == '.docx':
            loader = UnstructuredFileLoader(temp_file_path)
            document = loader.load()
            print(dir(document)) if document else print("No document loaded") ## Debugging output
            text_content = str(document)
        else:
            return JSONResponse(status_code=400, content={"message": "Unsupported file type"})
        result = text_check(text_content) # Sending the text to text checker API
        return JSONResponse(content=result)
    except Exception as e:
        return JSONResponse(status_code=500, content={"message": str(e)})
    finally:
        os.remove(temp_file_path)  # clean-up


if __name__ == "__main__":
    ngrok_tunnel = ngrok.connect(8000)
    print('Public URL:', ngrok_tunnel.public_url)
    nest_asyncio.apply()
    uvicorn.run(app, port=8000)

INFO:     Started server process [456]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://469e-34-106-89-170.ngrok-free.app
INFO:     111.68.99.94:0 - "GET / HTTP/1.1" 200 OK
INFO:     111.68.99.94:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     111.68.99.94:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     111.68.99.94:0 - "GET /openapi.json HTTP/1.1" 200 OK
['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pretty__', '__private_attributes__', '__pydantic_complete__', '__pydantic_core_schema__', '__pydantic_custom_init__', '__pydantic_decorators__', '__pydantic_extra__', '__pydantic_fields_set__', '__pydantic_generic_metadata__', '__

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [456]
